In [1]:
def is_left_turn( a, b, c):
    '''
    判断c是否在直线ab的左侧
    '''
    return (b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0]) > 0

def is_right_turn( a, b, c):
    '''
    判断p3是否在直线p1p2的右侧
    '''
    return (b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0]) < 0


# 凸包

In [2]:
import itertools
import sys
def is_left_turn( a, b, c):
    '''
    判断c是否在直线ab的左侧
    '''
    return (b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0]) > 0

def is_right_turn( a, b, c):
    '''
    判断p3是否在直线p1p2的右侧
    '''
    return (b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0]) < 0
def brute_force( points):
    hull = []
    n = len(points)

    def is_all_left(i, j):
        for k in points:
            if k != i and k != j:
                if not is_left_turn(i, j, k):
                    return False
        return True

    def is_all_right(i, j):
        for k in points:
            if k != i and k != j:
                if not  is_right_turn(i, j, k):
                    return False
        return True
    for i, j in itertools.combinations(points, 2):
        all_left = is_all_left(i, j)
        all_right = is_all_right(i, j)
        if all_left or all_right:
            hull.append((i, j))
    return hull

In [3]:

def divide_and_conquer( points):
    if len(points) <= 1:
        return points
    # 按照x 排序
    min_point = min(points, key=lambda p: p[0])
    max_point = max(points, key=lambda p: p[0])
    def distance(p, a, b):
        # 点p到直线ab的距离
        return abs((b[0] - a[0]) * (a[1] - p[1]) - (a[0] - p[0]) * (b[1] - a[1]))
    def find_hull(points, p1, p2):
        # 在p1p2左侧的点集找凸包
        if len(points)==0:
            return []
        farthest_point = max(points, key=lambda p: distance(p, p1, p2))
        left_set_p1_farthest = [
            p for p in points if is_left_turn(p1, farthest_point, p)]
        left_set_farthest_p2 = [
            p for p in points if is_left_turn(farthest_point, p2, p)]
        hull = find_hull(left_set_p1_farthest, p1, farthest_point) + \
            [farthest_point] + \
            find_hull(left_set_farthest_p2, farthest_point, p2)
        return hull
    left_set = [p for p in points if is_left_turn(
        min_point, max_point, p)]
    right_set = [p for p in points if is_left_turn(
        max_point, min_point, p)]

    hull = [min_point] + find_hull(left_set, min_point, max_point) + [
        max_point] + find_hull(right_set, max_point, min_point)
    return hull  # hull存储了最终凸包的所有点

In [4]:
from time import perf_counter_ns as pcns
import random

In [5]:
epochs = 10
num_points=[100,500,1000,5000,10000,50000,100000]
points_limt= [n*2 for n in num_points]
brute_force_time=[0]*len(num_points)
divide_and_conquer_time = [0]*len(num_points)

In [6]:
for epoch in range(epochs):
    print(f"{epoch=}")
    for i in range(len(num_points)):
        print(f"{num_points[i]=}",end='\n')
        points = [(random.randint(0, points_limt[i]), random.randint(0, points_limt[i]))
                for _ in range(num_points[i])]
        print(f'暴力法开始',end='\t')
        brute_force_start=pcns()
        brute_force(points)
        brute_force_end=pcns()
        print(f'暴力法结束',end='\t')
        brute_force_time[i]+=(brute_force_end-brute_force_start)

        print(f"分治法开始",end='\t')
        divide_and_conquer_start=pcns()
        divide_and_conquer(points)
        divide_and_conquer_end=pcns()
        print(f"分治法结束",)
        divide_and_conquer_time[i]+=(divide_and_conquer_end-divide_and_conquer_start)

brute_force_time=[t/epochs*1e-6 for t in brute_force_time]
divide_and_conquer_time=[t/epochs*1e-6 for t in divide_and_conquer_time]


epoch=0
num_points[i]=100
暴力法开始	暴力法结束	分治法开始	分治法结束
num_points[i]=500
暴力法开始	暴力法结束	分治法开始	分治法结束
num_points[i]=1000
暴力法开始	暴力法结束	分治法开始	分治法结束
num_points[i]=5000
暴力法开始	暴力法结束	分治法开始	分治法结束
num_points[i]=10000
暴力法开始	暴力法结束	分治法开始	分治法结束
num_points[i]=50000
暴力法开始	

In [ ]:
brute_force_time, divide_and_conquer_time

([0.055209999999999995, 7.9287, 801.9090199999999, 78078.90681999999],
 [0.01877, 0.16255, 1.50325, 13.60827])

In [ ]:
print(f"n\t\t\tbrute_force\t\t\tdivide_and_conquer")
for i in range(len(num_points)):
    print(
        f"{num_points[i]}\t\t\t{brute_force_time[i]:8f}\t\t\t{divide_and_conquer_time[i]:8f}")

n			brute_force			divide_and_conquer
10			0.055210			0.018770
100			7.928700			0.162550
1000			801.909020			1.503250
10000			78078.906820			13.608270
